# Assignment 01: Multi-class Classification 
In this Assignment, you will train a deep model on the CIFAR10 from the scratch using PyTorch.

### Basic Imports

In [1]:
import os
import time
import os.path as osp

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms
import torchvision

import matplotlib.pyplot as plt
from PIL import Image

### Hyperparameters

In [2]:
# random seed
SEED = 1 
NUM_CLASS = 10

# Training
BATCH_SIZE = 128
NUM_EPOCHS = 30
EVAL_INTERVAL=1
SAVE_DIR = './log'

# Optimizer
LEARNING_RATE = 1e-1
MOMENTUM = 0.9
STEP=5
GAMMA=0.5


In [3]:
### Device

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


### Dataset


In [5]:
# cifar10 transform
transform_cifar10_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_cifar10_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

train_set = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_cifar10_train)
#train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,shuffle=True, num_workers=2)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_cifar10_test)
#test_dataloader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


### Model

In [6]:
#class ConvNet(nn.Module):
    #def __init__(self):
        #super(ConvNet, self).__init__()
        #self.conv1 = nn.Conv2d(3, 4, 3)  
        #self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(4, 8, 3)  
        #self.fc1 = nn.Linear(8 * 6 * 6, 32)
        #self.fc2 = nn.Linear(32, 10)

    #def forward(self, x):
        #x = self.pool(torch.relu(self.conv1(x)))
        #x = self.pool(torch.relu(self.conv2(x)))
        #x = x.view(-1, 8 * 6 * 6)
        #x = torch.relu(self.fc1(x))
        #x = self.fc2(x)
        #return x
        
class MyModel(nn.Module):
    def __init__(self, num_classes):
    #def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.fc1 = nn.Linear(4096, num_classes)
        #self.fc1 = nn.Linear(in_features=6488, out_features=512)
        #self.fc1 = nn.Linear(8 * 6 * 6, 32)
        self.fc1 = nn.Linear(4096, 10)
        self.relu3 = nn.ReLU()
        self.drop1 = nn.Dropout(p=0.5)
        #self.fc2 = nn.Linear(in_features=512, out_features=num_classes)
        #self.fc2 = nn.Linear(32, 10)
        self.fc2 = nn.Linear(10, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 64*8*8)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.drop1(x)
        x = self.fc2(x)
        return x

In [7]:
#model = ConvNet()
model = MyModel(num_classes=10)
model = model.to(device)

### Optimizer

In [8]:
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP, gamma=GAMMA)

### Task 1: per batch training/testing
---

Please denfine two function named ``train_batch`` and ``test_batch``. These functions are essential for training and evaluating machine learning models using batched data from dataloaders.

**To do**: 
1. Define the loss function i.e [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).
2. Take the image as the input and generate the output using the pre-defined SimpleNet.
3. Calculate the loss between the output and the corresponding label using the loss function.

In [9]:
##################### Write your answer here ##################
# Define the loss function
#criterion = nn.CrossEntropyLoss()

class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
    
    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return torch.mean(focal_loss)

loss_functions = {
    'MAE': nn.L1Loss(),
    'CE': nn.CrossEntropyLoss(),
    'Focal (gamma=0.5)': FocalLoss(gamma=0.5),
    'Focal (gamma=2)': FocalLoss(gamma=2)
}

###############################################################

In [10]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        # Convert labels to one-hot encoding
        labels_onehot = F.one_hot(labels, num_classes=NUM_CLASS).float()
        loss = criterion(outputs, labels_onehot)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [11]:

def evaluate(model, criterion, test_loader):
    # TODO: Implement the evaluation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

### Model Training

In [12]:
for loss_name, loss_function in loss_functions.items():
    # Instantiate the model
    # Define the number of classes for the problem
    num_classes = 10 # change this to the appropriate value for your problem
    model = MyModel(num_classes)
    #model = MyModel()

    # Set up the optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    num_epochs = 10 # adjust as needed
    for epoch in range(num_epochs):
        train_loss = train(model, loss_function, optimizer, train_loader)
        print(f"Loss Function: {loss_name}, Epoch: {epoch+1}, Train Loss: {train_loss}")

    # Evaluate the model
    test_accuracy = evaluate(model, loss_function, test_loader)

    print(f"Loss Function: {loss_name}, Test Accuracy: {test_accuracy}")

Loss Function: MAE, Epoch: 1, Train Loss: 0.13510192747768537
Loss Function: MAE, Epoch: 2, Train Loss: 0.10011785128689787
Loss Function: MAE, Epoch: 3, Train Loss: 0.10011628742718026
Loss Function: MAE, Epoch: 4, Train Loss: 0.10010859231128717
Loss Function: MAE, Epoch: 5, Train Loss: 0.10011277237282995
Loss Function: MAE, Epoch: 6, Train Loss: 0.10010829091529407
Loss Function: MAE, Epoch: 7, Train Loss: 0.10011417942736155
Loss Function: MAE, Epoch: 8, Train Loss: 0.10010709522096702
Loss Function: MAE, Epoch: 9, Train Loss: 0.10010621763403764
Loss Function: MAE, Epoch: 10, Train Loss: 0.10010692006562982
Loss Function: MAE, Test Accuracy: 10.0
Loss Function: CE, Epoch: 1, Train Loss: 2.1763560021929726
Loss Function: CE, Epoch: 2, Train Loss: 2.0523084191715015
Loss Function: CE, Epoch: 3, Train Loss: 1.9731995656972041
Loss Function: CE, Epoch: 4, Train Loss: 1.937276638377353
Loss Function: CE, Epoch: 5, Train Loss: 1.9109439935220782
Loss Function: CE, Epoch: 6, Train Loss: